## Задание № 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [153]:
import pandas as pd
ratings = pd.read_csv('ml-latest-small/ratings.csv')

ratings_grouped = ratings.groupby('userId').count().reset_index()
ratings_filt = ratings_grouped.filter(items = ['userId', 'rating'])
need_users = ratings_filt[ratings_filt.rating > 100]
need_users.rename(columns={'rating': 'count_rating'}, inplace=True)

need_mean = ratings.groupby('userId').agg(['max', 'min'])['timestamp'].reset_index()
need_mean['mean time'] = need_mean['max'] - need_mean['min']
mean_filt = need_mean.filter(items = ['userId','mean time'])

results = need_users.merge(mean_filt)
results.head()

,userId,count_rating,mean time
0,4,204,203560
1,8,116,85187
2,15,1700,471393496
3,17,363,8053
4,19,423,5282


## Задание № 2 

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [14]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [15]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [16]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [17]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;
- аналогичную таблицу по типам выручки с указанием адреса клиента.

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [20]:
data = (
    rzd
    .merge(auto, how = 'outer')
    .merge(air, how = 'outer')
)

data

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [22]:
data.merge(client_base, how = 'outer')

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


## Задание № 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

1) У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

2) Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

3) Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Данную проблему можно легко решить с помощью библиотеки pandas, воспользовшись группировкой и любыми двумя агрегатными функциями для информации о координатах пользователей info_users.groupby('userId').agg('max', 'min')'coordinates'.reset_index() или сводной таблицей и любыми двумя агрегатными функциями info_users.pivot_table(index = 'userId', columns = 'count', values = 'coordinates', aggfunc = 'count', fill_value = 0). После чего объединил эти данные с помощью метода joined = info_users.merge(coordinates, on='Id', how='inner').

Эта информация позволяет определить местоположение основной массы покупателей и расстояние от них до магазина, что даёт ответы на следующий вопросы: в какие магазины привезти товара побольше, какие места для продажи окупаются и будет ли выгодна доставка товаров онлайн.